In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Install SAM 3 from Drive
import os
import sys

SAM3_PATH = "/content/drive/MyDrive/sam3"

if not os.path.exists(SAM3_PATH):
    print("Cloning SAM 3...")
    !git clone https://github.com/facebookresearch/sam3.git "{SAM3_PATH}"

%cd "{SAM3_PATH}"
!pip install -e .
%cd /content

# Install dependencies - decord needs special handling
!pip install -q pillow-heif triton ultralytics
!pip install -q decord
# If decord fails, try av as fallback
try:
    import decord
    print("✓ decord installed")
except:
    print("decord failed, installing av as fallback...")
    !pip install -q av

!pip install -q ultralytics
print("✓ ultralytics installed")

print("✅ Installation complete!")

In [ ]:
# Imports i setup
import torch
import numpy as np
import cv2
import json
from PIL import Image
import pillow_heif
import matplotlib.pyplot as plt

pillow_heif.register_heif_opener()

# Add SAM3 to path
SAM3_PATH = "/content/drive/MyDrive/sam3"
if SAM3_PATH not in sys.path:
    sys.path.insert(0, SAM3_PATH)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")
if device == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True
    torch.autocast("cuda", dtype=torch.bfloat16).__enter__()

In [ ]:
# Hugging Face login (required for SAM 3)
from huggingface_hub import login
login()

In [ ]:
# Load SAM 3 model
from sam3 import build_sam3_image_model
from sam3.model.sam3_image_processor import Sam3Processor

bpe_path = f"{SAM3_PATH}/sam3/assets/bpe_simple_vocab_16e6.txt.gz"

print("Loading SAM 3 model...")
model = build_sam3_image_model(bpe_path=bpe_path, device=device)

CONFIDENCE_THRESHOLD = 0.15
processor = Sam3Processor(model, confidence_threshold=CONFIDENCE_THRESHOLD)
print(f"✓ SAM 3 loaded (threshold: {CONFIDENCE_THRESHOLD})")

In [ ]:
# ============================================================
# KONFIGURACIJA
# ============================================================

# Input/Output paths
DRIVE_PATH = "/content/drive/MyDrive/TRAIN"
OUTPUT_DIR = "/content/drive/MyDrive/segmented_output_improved"

# YOLO weights for facade detection
YOLO_WEIGHTS = "/content/drive/MyDrive/best.pt"

# Temporary folder for facade crops (will be created automatically)
FACADE_DIR = "/content/drive/MyDrive/segmented_output_improved/facade_crops"

# Component prompts - više varijacija = bolja detekcija
COMPONENT_PROMPTS = {
    "seal": [
        "seal", "rubber seal", "black rubber seal", "gasket",
        "rubber gasket", "weatherstrip", "window gasket",
        "black rubber strip", "rubber edging", "edge seal",
        "seal around window", "rubber seal around glass"
    ],
    "tin": [
        "tin", "metal tin", "metal sheet", "metal casing",
        "aluminum panel", "metal panel", "sheet metal"
    ],
    "screw": [
        "screw", "small screw", "screw head", "fastener",
        "bolt", "metal screw", "fixing screw"
    ],
    "hole": [
        "hole", "circular hole", "opening", "perforation",
        "vent hole", "round hole", "drill hole"
    ],
    "glass": [
        "glass", "window glass", "transparent pane",
        "glass panel", "window pane", "clear glass"
    ]
}

# Boje za vizualizaciju
COMPONENT_COLORS = {
    "seal": (0, 0, 255),      # Plava
    "tin": (0, 255, 0),       # Zelena
    "screw": (255, 0, 0),     # Crvena
    "hole": (255, 255, 0),    # Žuta
    "glass": (255, 0, 255)    # Magenta
}

os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(FACADE_DIR, exist_ok=True)
print(f"✓ Output dir: {OUTPUT_DIR}")
print(f"✓ Facade crops dir: {FACADE_DIR}")
print(f"✓ Components: {list(COMPONENT_PROMPTS.keys())}")

## YOLO Facade Detection (Built-in)
This section detects and crops facades from raw images - no need to run separate YOLO notebook!

In [ ]:
# ============================================================
# YOLO FACADE DETECTION
# Detects facades in raw images and saves crops
# ============================================================

from ultralytics import YOLO

def detect_and_crop_facades(source_dir, output_dir, yolo_weights, confidence=0.25):
    """
    Run YOLO to detect facades and save cropped regions.
    
    Args:
        source_dir: Path to TRAIN folder with positive/negative subfolders
        output_dir: Where to save facade crops (maintains folder structure)
        yolo_weights: Path to YOLO weights (best.pt)
        confidence: Minimum confidence threshold
    
    Returns:
        dict: Mapping of original image path -> cropped facade path
    """
    print(f"\n{'='*60}")
    print("YOLO FACADE DETECTION")
    print(f"{'='*60}")
    
    # Load YOLO model
    if not os.path.exists(yolo_weights):
        print(f"ERROR: YOLO weights not found: {yolo_weights}")
        return {}
    
    model = YOLO(yolo_weights)
    print(f"✓ YOLO model loaded: {yolo_weights}")
    print(f"  Classes: {model.names}")
    
    # Facade class detection
    facade_aliases = {"facade", "fascade", "facade_element", "facadeelement"}
    facade_ids = []
    if model.names:
        for cls_id, name in model.names.items():
            if str(name).lower() in facade_aliases:
                facade_ids.append(int(cls_id))
        # Single-class model = use that class
        if not facade_ids and len(model.names) == 1:
            facade_ids = [int(list(model.names.keys())[0])]
    
    print(f"  Facade class IDs: {facade_ids}")
    
    image_extensions = ('.jpg', '.jpeg', '.png', '.heic', '.heif')
    crop_mapping = {}
    
    for folder in ['positive', 'negative']:
        folder_path = os.path.join(source_dir, folder)
        if not os.path.exists(folder_path):
            print(f"  Skipping {folder}/ - not found")
            continue
        
        output_folder = os.path.join(output_dir, folder)
        os.makedirs(output_folder, exist_ok=True)
        
        images = [f for f in os.listdir(folder_path) 
                  if f.lower().endswith(image_extensions) and not f.startswith('.')]
        
        print(f"\n--- {folder}/ ({len(images)} images) ---")
        
        for img_file in images:
            img_path = os.path.join(folder_path, img_file)
            
            try:
                # Load image via PIL (supports HEIC)
                pil_img = Image.open(img_path)
                if pil_img.mode != 'RGB':
                    pil_img = pil_img.convert('RGB')
                img_bgr = cv2.cvtColor(np.array(pil_img), cv2.COLOR_RGB2BGR)
                h, w = img_bgr.shape[:2]
                
                # Run YOLO
                results = model(img_bgr, verbose=False)
                
                # Find best facade detection
                best_box = None
                best_conf = -1.0
                best_cls = None
                
                for r in results:
                    boxes = getattr(r, 'boxes', None)
                    if boxes is None:
                        continue
                    
                    for box in boxes:
                        conf = float(box.conf[0])
                        cls_id = int(box.cls[0])
                        
                        if facade_ids and cls_id not in facade_ids:
                            continue
                        
                        if conf >= confidence and conf > best_conf:
                            best_conf = conf
                            best_box = box
                            best_cls = cls_id
                
                if best_box is None:
                    print(f"  {img_file}: No facade detected")
                    continue
                
                # Crop facade
                x1, y1, x2, y2 = best_box.xyxy[0].cpu().numpy()
                x1, y1, x2, y2 = int(max(0, x1)), int(max(0, y1)), int(min(w, x2)), int(min(h, y2))
                
                crop = img_bgr[y1:y2, x1:x2]
                if crop.size == 0:
                    print(f"  {img_file}: Empty crop")
                    continue
                
                # Save crop
                base_name = os.path.splitext(img_file)[0].replace(" ", "_")
                save_name = f"{base_name}_facade.jpg"
                save_path = os.path.join(output_folder, save_name)
                
                cv2.imwrite(save_path, crop)
                crop_mapping[img_path] = save_path
                print(f"  {img_file}: ✓ facade (conf={best_conf:.2f})")
                
            except Exception as e:
                print(f"  {img_file}: Error - {e}")
    
    print(f"\n✓ Detected {len(crop_mapping)} facades")
    print(f"✓ Saved to: {output_dir}")
    
    return crop_mapping


# Run YOLO facade detection
print("Running YOLO facade detection on TRAIN images...")
facade_crops = detect_and_crop_facades(
    source_dir=DRIVE_PATH,
    output_dir=FACADE_DIR,
    yolo_weights=YOLO_WEIGHTS,
    confidence=0.25
)
print(f"\n✅ YOLO facade detection complete! {len(facade_crops)} facades ready for SAM segmentation.")

## Core Functions

In [ ]:
# ============================================================
# CORE FUNCTIONS
# ============================================================

def load_image(image_path):
    """Load image (HEIC, JPG, PNG supported)"""
    try:
        img = Image.open(image_path)
        if img.mode != 'RGB':
            img = img.convert('RGB')
        return img
    except Exception as e:
        print(f"Error loading {image_path}: {e}")
        return None


def segment_with_text_prompt(pil_image, text_prompt):
    """Segment using SAM 3 text prompt"""
    try:
        inference_state = processor.set_image(pil_image)
        output = processor.set_text_prompt(
            state=inference_state,
            prompt=text_prompt
        )
        
        masks = output.get("masks", None)
        boxes = output.get("boxes", None)
        scores = output.get("scores", None)
        
        if masks is not None and len(masks) > 0:
            return masks, boxes, scores
        return None, None, None
    except Exception as e:
        return None, None, None


def segment_with_multiple_prompts(pil_image, prompts_list, min_score=0.1):
    """Try multiple prompts and combine results"""
    all_masks = []
    all_boxes = []
    all_scores = []
    all_prompts = []
    
    for prompt in prompts_list:
        masks, boxes, scores = segment_with_text_prompt(pil_image, prompt)
        
        if masks is not None and len(masks) > 0:
            if scores is not None:
                for i in range(len(masks)):
                    score = scores[i].item() if hasattr(scores[i], 'item') else float(scores[i])
                    if score >= min_score:
                        all_masks.append(masks[i])
                        all_boxes.append(boxes[i] if boxes is not None else None)
                        all_scores.append(score)
                        all_prompts.append(prompt)
            else:
                for i in range(len(masks)):
                    all_masks.append(masks[i])
                    all_boxes.append(boxes[i] if boxes is not None else None)
                    all_scores.append(1.0)
                    all_prompts.append(prompt)
    
    return all_masks, all_boxes, all_scores, all_prompts

## Pixel-Level Mask Functions (NO bounding box!)

In [ ]:
# ============================================================
# PIXEL-LEVEL MASK FUNCTIONS
# Ove funkcije ekstrahiraju SAMO piksele unutar maske!
# ============================================================

def extract_masked_region(image_np, mask, background_color=(0, 0, 0)):
    """
    Ekstrahira SAMO piksele unutar maske.
    Okolina postaje crna (ili zadana boja).
    
    Returns:
        masked_image: Slika gdje su samo pikseli unutar maske vidljivi
        mask_bool: Boolean maska
    """
    if isinstance(mask, torch.Tensor):
        mask = mask.cpu().numpy()
    
    mask_bool = mask.astype(bool)
    if mask_bool.ndim > 2:
        mask_bool = mask_bool.squeeze()
    
    # Stvori praznu sliku (crna pozadina)
    masked_image = np.full_like(image_np, background_color)
    
    # Kopiraj SAMO piksele unutar maske
    masked_image[mask_bool] = image_np[mask_bool]
    
    return masked_image, mask_bool


def extract_masked_region_rgba(image_np, mask):
    """
    Ekstrahira regiju s TRANSPARENTNOM pozadinom (RGBA).
    Savršeno za overlay bez okoline.
    """
    if isinstance(mask, torch.Tensor):
        mask = mask.cpu().numpy()
    
    mask_bool = mask.astype(bool)
    if mask_bool.ndim > 2:
        mask_bool = mask_bool.squeeze()
    
    h, w = image_np.shape[:2]
    rgba = np.zeros((h, w, 4), dtype=np.uint8)
    rgba[:, :, :3] = image_np
    rgba[:, :, 3] = (mask_bool * 255).astype(np.uint8)
    
    return rgba, mask_bool


def get_tight_crop_with_mask(image_np, mask, padding=10):
    """
    Tight crop oko maske, ALI pikseli izvan maske su crni.
    
    Returns:
        masked_crop: Cropped slika s maskom primijenjenom
        cropped_mask: Cropped boolean maska
    """
    if isinstance(mask, torch.Tensor):
        mask = mask.cpu().numpy()
    
    mask_bool = mask.astype(bool)
    if mask_bool.ndim > 2:
        mask_bool = mask_bool.squeeze()
    
    # Nađi bounding box maske
    rows = np.any(mask_bool, axis=1)
    cols = np.any(mask_bool, axis=0)
    
    if not rows.any() or not cols.any():
        return None, None
    
    y_min, y_max = np.where(rows)[0][[0, -1]]
    x_min, x_max = np.where(cols)[0][[0, -1]]
    
    # Dodaj padding
    h, w = image_np.shape[:2]
    y_min = max(0, y_min - padding)
    y_max = min(h, y_max + padding)
    x_min = max(0, x_min - padding)
    x_max = min(w, x_max + padding)
    
    # Crop
    cropped_image = image_np[y_min:y_max, x_min:x_max].copy()
    cropped_mask = mask_bool[y_min:y_max, x_min:x_max]
    
    # Primijeni masku - pikseli izvan su crni
    masked_crop = cropped_image.copy()
    masked_crop[~cropped_mask] = 0
    
    return masked_crop, cropped_mask


print("✓ Pixel-level mask functions loaded")

## Mask Processing & Refinement

In [ ]:
# ============================================================
# MASK PROCESSING & REFINEMENT
# ============================================================

def refine_mask_morphology(mask, kernel_size=5, iterations=2):
    """
    Poboljšaj masku morfološkim operacijama.
    - Closing: zatvara male rupe
    - Opening: uklanja šum
    """
    if isinstance(mask, torch.Tensor):
        mask = mask.cpu().numpy()
    
    mask_uint8 = (mask.astype(bool) * 255).astype(np.uint8)
    if mask_uint8.ndim > 2:
        mask_uint8 = mask_uint8.squeeze()
    
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kernel_size, kernel_size))
    
    # Closing - zatvara male rupe
    closed = cv2.morphologyEx(mask_uint8, cv2.MORPH_CLOSE, kernel, iterations=iterations)
    
    # Opening - uklanja šum
    opened = cv2.morphologyEx(closed, cv2.MORPH_OPEN, kernel, iterations=1)
    
    return opened > 0


def combine_overlapping_masks(masks, iou_threshold=0.5):
    """
    Kombiniraj maske koje se preklapaju (duplikati).
    """
    if len(masks) <= 1:
        return masks
    
    mask_arrays = []
    for m in masks:
        if isinstance(m, torch.Tensor):
            m = m.cpu().numpy()
        if m.ndim > 2:
            m = m.squeeze()
        mask_arrays.append(m.astype(bool))
    
    combined = []
    used = set()
    
    for i, m1 in enumerate(mask_arrays):
        if i in used:
            continue
        
        current_mask = m1.copy()
        used.add(i)
        
        for j, m2 in enumerate(mask_arrays):
            if j <= i or j in used:
                continue
            
            intersection = np.logical_and(current_mask, m2).sum()
            union = np.logical_or(current_mask, m2).sum()
            iou = intersection / union if union > 0 else 0
            
            if iou > iou_threshold:
                current_mask = np.logical_or(current_mask, m2)
                used.add(j)
        
        combined.append(current_mask)
    
    return combined


def filter_masks_by_size(masks, min_area=100, max_area_ratio=0.9):
    """
    Filtriraj maske po veličini.
    - Ukloni premale (šum)
    - Ukloni prevelike (cijela slika)
    """
    filtered = []
    for m in masks:
        if isinstance(m, torch.Tensor):
            m = m.cpu().numpy()
        if m.ndim > 2:
            m = m.squeeze()
        
        area = m.sum()
        total = m.size
        
        if area >= min_area and area / total <= max_area_ratio:
            filtered.append(m)
    
    return filtered


print("✓ Mask processing functions loaded")

## Visualization & Export

In [ ]:
# ============================================================
# VISUALIZATION & EXPORT
# ============================================================

def create_component_overlay(image_np, all_results, alpha=0.5):
    """
    Stvori overlay s pixel maskama (ne bounding boxovima!).
    """
    overlay = image_np.copy().astype(np.float32)
    
    for component, data in all_results.items():
        masks = data.get('masks', [])
        color = np.array(COMPONENT_COLORS.get(component, (128, 128, 128)))
        
        for mask in masks:
            if isinstance(mask, torch.Tensor):
                mask = mask.cpu().numpy()
            
            mask_bool = mask.astype(bool)
            if mask_bool.ndim > 2:
                mask_bool = mask_bool.squeeze()
            
            # Primijeni boju SAMO na piksele unutar maske
            overlay[mask_bool] = overlay[mask_bool] * (1 - alpha) + color * alpha
    
    return overlay.astype(np.uint8)


def create_mask_contours(image_np, all_results, thickness=2):
    """
    Nacrtaj samo konture maski (bez fill-a).
    """
    result = image_np.copy()
    
    for component, data in all_results.items():
        masks = data.get('masks', [])
        scores = data.get('scores', [])
        color = COMPONENT_COLORS.get(component, (128, 128, 128))
        
        for i, mask in enumerate(masks):
            if isinstance(mask, torch.Tensor):
                mask = mask.cpu().numpy()
            
            mask_bool = mask.astype(bool)
            if mask_bool.ndim > 2:
                mask_bool = mask_bool.squeeze()
            
            # Nacrtaj konturu
            contours, _ = cv2.findContours(
                mask_bool.astype(np.uint8),
                cv2.RETR_EXTERNAL,
                cv2.CHAIN_APPROX_SIMPLE
            )
            cv2.drawContours(result, contours, -1, color, thickness)
            
            # Label
            if len(contours) > 0:
                M = cv2.moments(contours[0])
                if M["m00"] > 0:
                    cx = int(M["m10"] / M["m00"])
                    cy = int(M["m01"] / M["m00"])
                    score = scores[i] if i < len(scores) else 0
                    if hasattr(score, 'item'):
                        score = score.item()
                    label = f"{component}:{score:.2f}"
                    cv2.putText(result, label, (cx-30, cy),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255,255,255), 2)
                    cv2.putText(result, label, (cx-30, cy),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.4, color, 1)
    
    return result


def save_individual_components(image_np, all_results, output_dir, base_name):
    """
    Spremi svaku komponentu zasebno (bez okoline!).
    """
    os.makedirs(output_dir, exist_ok=True)
    saved_files = []
    
    for component, data in all_results.items():
        masks = data.get('masks', [])
        scores = data.get('scores', [])
        
        for i, mask in enumerate(masks):
            masked_crop, _ = get_tight_crop_with_mask(image_np, mask)
            
            if masked_crop is None:
                continue
            
            score = scores[i] if i < len(scores) else 0.0
            if hasattr(score, 'item'):
                score = score.item()
            
            filename = f"{base_name}_{component}_{i+1}_score{score:.2f}.png"
            filepath = os.path.join(output_dir, filename)
            
            cv2.imwrite(filepath, cv2.cvtColor(masked_crop, cv2.COLOR_RGB2BGR))
            saved_files.append(filepath)
    
    return saved_files


def save_results_json(all_results, output_path, image_name, image_size):
    """
    Spremi JSON s informacijama o maskama.
    """
    json_data = {
        'image': image_name,
        'size': list(image_size),
        'components': {}
    }
    
    for component, data in all_results.items():
        masks = data.get('masks', [])
        scores = data.get('scores', [])
        
        component_info = []
        for i, mask in enumerate(masks):
            mask_bool = mask if isinstance(mask, np.ndarray) else mask.cpu().numpy()
            if mask_bool.ndim > 2:
                mask_bool = mask_bool.squeeze()
            
            area = int(mask_bool.sum())
            coverage = float(area / mask_bool.size)
            
            # Bounding box
            rows = np.any(mask_bool, axis=1)
            cols = np.any(mask_bool, axis=0)
            if rows.any() and cols.any():
                y_min, y_max = int(np.where(rows)[0][0]), int(np.where(rows)[0][-1])
                x_min, x_max = int(np.where(cols)[0][0]), int(np.where(cols)[0][-1])
                bbox = [x_min, y_min, x_max, y_max]
            else:
                bbox = None
            
            score = scores[i] if i < len(scores) else 1.0
            if hasattr(score, 'item'):
                score = score.item()
            
            component_info.append({
                'index': i,
                'area_pixels': area,
                'coverage_percent': round(coverage * 100, 2),
                'bbox': bbox,
                'score': round(float(score), 3)
            })
        
        json_data['components'][component] = component_info
    
    with open(output_path, 'w') as f:
        json.dump(json_data, f, indent=2)


print("✓ Visualization & export functions loaded")

## Main Processing Function

In [ ]:
# ============================================================
# MAIN PROCESSING FUNCTION
# ============================================================

def process_image(image_path, output_dir, save_components=True):
    """
    Procesiraj jednu sliku:
    1. Segmentiraj sve komponente
    2. Refiniraj maske (morfologija + dedup)
    3. Spremi overlay, komponente, JSON
    """
    print(f"\n{'='*60}")
    print(f"Processing: {os.path.basename(image_path)}")
    print(f"{'='*60}")
    
    # Load
    pil_image = load_image(image_path)
    if pil_image is None:
        return None, None
    
    # Resize ako je preveliko
    max_dim = 1536
    w, h = pil_image.size
    if max(h, w) > max_dim:
        scale = max_dim / max(h, w)
        pil_image = pil_image.resize((int(w*scale), int(h*scale)), Image.LANCZOS)
        print(f"  Resized: {w}x{h} → {pil_image.size}")
    
    image_np = np.array(pil_image)
    base_name = os.path.splitext(os.path.basename(image_path))[0].replace(" ", "_")
    
    os.makedirs(output_dir, exist_ok=True)
    
    all_results = {}
    total_detections = 0
    
    # Segment svaku komponentu
    for component, prompts in COMPONENT_PROMPTS.items():
        print(f"  [{component}] ", end="")
        
        masks, boxes, scores, used_prompts = segment_with_multiple_prompts(
            pil_image, prompts, min_score=0.1
        )
        
        if masks and len(masks) > 0:
            # 1. Refiniraj morfologijom
            refined_masks = [refine_mask_morphology(m, kernel_size=3) for m in masks]
            
            # 2. Filtriraj po veličini
            filtered_masks = filter_masks_by_size(refined_masks, min_area=100)
            
            # 3. Kombiniraj preklapajuće
            combined_masks = combine_overlapping_masks(filtered_masks, iou_threshold=0.3)
            
            print(f"✓ {len(masks)} → {len(combined_masks)} masks")
            total_detections += len(combined_masks)
            
            all_results[component] = {
                'masks': combined_masks,
                'scores': scores[:len(combined_masks)] if scores else [1.0] * len(combined_masks)
            }
        else:
            print(f"✗ none")
    
    print(f"\n  TOTAL: {total_detections} components")
    
    # Save results
    # 1. Overlay
    overlay = create_component_overlay(image_np, all_results, alpha=0.4)
    overlay_path = os.path.join(output_dir, f"{base_name}_overlay.jpg")
    cv2.imwrite(overlay_path, cv2.cvtColor(overlay, cv2.COLOR_RGB2BGR))
    
    # 2. Contours only
    contours_img = create_mask_contours(image_np, all_results)
    contours_path = os.path.join(output_dir, f"{base_name}_contours.jpg")
    cv2.imwrite(contours_path, cv2.cvtColor(contours_img, cv2.COLOR_RGB2BGR))
    
    # 3. Individual components
    if save_components:
        comp_dir = os.path.join(output_dir, "components", base_name)
        saved = save_individual_components(image_np, all_results, comp_dir, base_name)
        print(f"  Saved {len(saved)} component crops")
    
    # 4. JSON
    json_path = os.path.join(output_dir, f"{base_name}_masks.json")
    save_results_json(all_results, json_path, os.path.basename(image_path), pil_image.size)
    
    print(f"  ✓ Saved to: {output_dir}")
    
    return all_results, overlay


print("✓ Main processing function ready")

## Test on Single Image

In [ ]:
# Test na jednoj slici
test_folder = FACADE_DIR + "/positive"  # Uses our own YOLO facade crops

if os.path.exists(test_folder):
    test_images = [f for f in os.listdir(test_folder) 
                   if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    
    if test_images:
        test_path = os.path.join(test_folder, test_images[0])
        output_test = os.path.join(OUTPUT_DIR, "test")
        
        results, overlay = process_image(test_path, output_test)
        
        # Load and resize original to match mask dimensions
        orig_pil = load_image(test_path)
        max_dim = 1536
        w, h = orig_pil.size
        if max(h, w) > max_dim:
            scale = max_dim / max(h, w)
            orig_pil = orig_pil.resize((int(w*scale), int(h*scale)), Image.LANCZOS)
        orig = np.array(orig_pil)
        
        # Prikaži rezultate
        fig, axes = plt.subplots(1, 3, figsize=(18, 6))
        
        # Original
        axes[0].imshow(orig)
        axes[0].set_title("Original")
        axes[0].axis('off')
        
        # Overlay
        axes[1].imshow(overlay)
        axes[1].set_title("Overlay (pixel masks)")
        axes[1].axis('off')
        
        # Contours
        contours = create_mask_contours(orig, results)
        axes[2].imshow(contours)
        axes[2].set_title("Contours only")
        axes[2].axis('off')
        
        plt.tight_layout()
        plt.show()
        
        # Prikaži pojedinačne komponente
        print("\n=== Individual Components (no background!) ===")
        for comp, data in results.items():
            if data['masks']:
                num_masks = min(len(data['masks']), 5)
                print(f"\n{comp.upper()}: {len(data['masks'])} masks")
                
                fig, axes = plt.subplots(1, num_masks, figsize=(3*num_masks, 3))
                if num_masks == 1:
                    axes = [axes]
                
                for i, mask in enumerate(data['masks'][:5]):
                    try:
                        crop, _ = get_tight_crop_with_mask(orig, mask)
                        if crop is not None:
                            axes[i].imshow(crop)
                            score = data['scores'][i] if i < len(data['scores']) else 0
                            axes[i].set_title(f"Score: {score:.2f}")
                            axes[i].axis('off')
                    except Exception as e:  
                        print(f"  Error showing {comp} #{i}: {e}")
                        axes[i].set_title("Error")
                        axes[i].axis('off')
                
                plt.tight_layout()
                plt.show()
    else:
        print("No test images found! Run YOLO facade detection first (cell above).")
else:
    print(f"Test folder not found: {test_folder}")
    print("Run YOLO facade detection first (cell above)!")

## Process All Images

In [ ]:
def process_folder(folder_path, output_folder):
    """Process all images in folder"""
    extensions = ('.jpg', '.jpeg', '.png', '.heic', '.heif')
    images = [f for f in os.listdir(folder_path)
              if f.lower().endswith(extensions) and not f.startswith('.')]
    
    print(f"\n{'#'*60}")
    print(f"Processing {len(images)} images from {folder_path}")
    print(f"{'#'*60}")
    
    for i, img_file in enumerate(images):
        print(f"\n[{i+1}/{len(images)}]", end="")
        
        input_path = os.path.join(folder_path, img_file)
        
        try:
            process_image(input_path, output_folder, save_components=True)
        except Exception as e:
            print(f"  Error: {e}")
    
    print(f"\n{'#'*60}")
    print(f"Done! Results: {output_folder}")


# Process positive folder (using our YOLO facade crops)
positive_path = FACADE_DIR + "/positive"
if os.path.exists(positive_path):
    process_folder(positive_path, os.path.join(OUTPUT_DIR, "positive"))
else:
    print(f"Positive folder not found: {positive_path}")
    print("Run YOLO facade detection cell first!")

In [ ]:
# Process negative folder (using our YOLO facade crops)
negative_path = FACADE_DIR + "/negative"
if os.path.exists(negative_path):
    process_folder(negative_path, os.path.join(OUTPUT_DIR, "negative"))
else:
    print(f"Negative folder not found: {negative_path}")
    print("Run YOLO facade detection cell first!")

## Done! 🎉

This notebook is **fully independent** - it includes:
1. **YOLO facade detection** - crops facades from raw TRAIN images
2. **SAM3 segmentation** - extracts pixel-level masks for all components

Results saved to: `/content/drive/MyDrive/segmented_output_improved/`

**Output structure:**
```
segmented_output_improved/
├── facade_crops/           # YOLO facade crops (intermediate)
│   ├── positive/
│   └── negative/
├── positive/
│   ├── IMG_xxx_overlay.jpg      # Vizualizacija s maskama
│   ├── IMG_xxx_contours.jpg     # Samo konture
│   ├── IMG_xxx_masks.json       # Metadata (area, bbox, coverage%)
│   └── components/
│       └── IMG_xxx/
│           ├── IMG_xxx_tin_1_score0.85.png    # Izolirane komponente
│           ├── IMG_xxx_glass_1_score0.92.png
│           └── ...
└── negative/
    └── ...
```

**No need to run YOLO_SAM_Facade_Colab.ipynb separately!**